In [1]:
# Importing Libraries
import sqlite3
import pandas as pd
import source_to_landing
import landing_to_sql

In [5]:
# Functions
def query_db(sql_query):
    connection = sqlite3.connect("database/test.db")
    cursor = connection.cursor()
    cursor.execute(sql_query)
    return cursor.fetchall()

In [6]:
query = """
SELECT * FROM example;

"""

In [7]:
query_db(query)

[('2024-04-15DL1075None',
  '2024-04-15',
  'landed',
  '1075',
  'DL1075',
  'DAL1075',
  'Delta Air Lines',
  'DL',
  'DAL',
  'Pittsburgh International',
  'America/New_York',
  'PIT',
  'KPIT',
  '',
  'D76',
  '18',
  '2024-04-15T13:01:00+00:00',
  '2024-04-15T13:01:00+00:00',
  '2024-04-15T13:18:00+00:00',
  '2024-04-15T13:18:00+00:00',
  '2024-04-15T13:18:00+00:00',
  'Hartsfield-jackson Atlanta International',
  'America/New_York',
  'ATL',
  'KATL',
  'S',
  'B29',
  '4',
  '',
  '2024-04-15T14:45:00+00:00',
  '2024-04-15T14:45:00+00:00',
  '2024-04-15T14:28:00+00:00',
  '2024-04-15T14:28:00+00:00',
  '2024-04-15T14:28:00+00:00',
  'None',
  'None',
  'None',
  'None'),
 ('2024-04-15DL5006None',
  '2024-04-15',
  'landed',
  '5006',
  'DL5006',
  'DAL5006',
  'Delta Air Lines',
  'DL',
  'DAL',
  'Raleigh-durham International Airport',
  'America/New_York',
  'RDU',
  'KRDU',
  '2',
  'D14',
  '22',
  '2024-04-15T06:15:00+00:00',
  '2024-04-15T06:15:00+00:00',
  '2024-04-15T06

In [ ]:
paath = source_to_landing.retrieve_hist_flights("2024-04-17","dl","iata")
landing_to_sql.upload_csv(paath)